# A simple classification model using Keras with Cloud TPUs

:) Away due holidays don't have any deep learning rig nearby

###Imports

In [0]:
#  Copyright 2019 Kiril Cvetkov. All Rights Reserved.


import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import *

In [0]:
print(tf.__version__)

1.12.0


###Resolve TPU Address

###Training of the model on TPU

In [0]:
class ConvBnAct(tf.keras.Model):
  
  def __init__(self, n_filters=64, kernel=(2,2), strides=(1,1)):
    self.filters = n_filters
    self.kernel = kernel
    self.strides = strides
    self.activation = tf.nn.relu
    super(ConvBnAct, self).__init__(name='')
    
    self.conv_ =  Conv2D(filters=self.filters,
                  kernel_size = self.kernel,
                  strides = self.strides, name='Conv2D')
    
    self.batch_norm = BatchNormalization(name='BatchNorm')
    
    self.activation = Activation(self.activation, name='RELU')
  def call(self, x, training=False):
    x = self.conv_(x) 
    x = self.batch_norm(x)
    
    x = self.activation(x)
    
    return x 
  

In [0]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = ConvBnAct()
block(norm)

print([x.name for x in block.variables])
    
  

['conv_bn_act/Conv2D/kernel:0', 'conv_bn_act/Conv2D/bias:0', 'conv_bn_act/BatchNorm/gamma:0', 'conv_bn_act/BatchNorm/beta:0', 'conv_bn_act/BatchNorm/moving_mean:0', 'conv_bn_act/BatchNorm/moving_variance:0']


In [0]:
class ConvAct(tf.keras.Model):
    def __init__(self, n_filters, kernel=(1,1), activation = tf.nn.relu, pooling=False):
      super(ConvAct, self).__init__(name='')

      self.pooling = pooling
      self.kernel = kernel
      self.n_filters = n_filters
      self.activation = activation
      self.pooling = pooling


      self.poolingLayer = AveragePooling2D(pool_size=(1,1), padding='same', name='AveragePool')
      self.convLayer = Conv2D(filters = self.n_filters,
                         kernel_size = self.kernel,
                         strides=1, name='Conv2D')

      self.activation = Activation(activation, name='ActivationRelu')

    def call(self, x, training = False):

      if self.pooling:
        x = self.poolingLayer(x)

      x = self.convLayer(x)
      x = self.activation(x)

      return x

   

In [0]:
norm = tf.random_normal([10, 127, 127, 64], mean=-1, stddev=4)

block = ConvAct(n_filters=64, pooling=True)
block(norm)

print([x.name for x in block.variables])

['conv_act/Conv2D/kernel:0', 'conv_act/Conv2D/bias:0']


In [0]:
class AttentionRefinmentModule(tf.keras.Model):
  def __init__(self, n_filters):
    super(AttentionRefinmentModule, self).__init__(name='')

    self.filters = n_filters
    
    self.poolingLayer = AveragePooling2D(pool_size = (1,1), padding='same', name='AveragePool')
    self.conv_bn_act = ConvBnAct(kernel = (1,1), n_filters = self.filters)
    
    
  def call(self, inputs, training = False):
    x = self.poolingLayer(inputs)
    x = self.conv_bn_act(x)
    
    print(x.shape)
    print(inputs.shape)
    return tf.math.multiply(inputs,x)

    

In [0]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = AttentionRefinmentModule(n_filters=64)
block(norm)

print([x.name for x in block.variables])

(10, 256, 256, 64)
(10, 256, 256, 64)
['attention_refinment_module/conv_bn_act_1/Conv2D/kernel:0', 'attention_refinment_module/conv_bn_act_1/Conv2D/bias:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/gamma:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/beta:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/moving_mean:0', 'attention_refinment_module/conv_bn_act_1/BatchNorm/moving_variance:0']


In [0]:
class FeatureFusionModule(tf.keras.Model):
  def __init__(self, n_filters):
    super(FeatureFusionModule, self).__init__(name='')
    self.conv_bn_act = ConvBnAct(n_filters=n_filters, kernel=(3, 3), strides=2)
    self.conv_act1 = ConvAct(n_filters=n_filters, pooling=True)
    self.conv_act2 = ConvAct(n_filters=n_filters, pooling=False, activation = tf.nn.sigmoid)
    activation = tf.nn.sigmoid
    
   
  def call(self, input_f, input_s, training=True):
    
    print(input_f.shape, input_s.shape)
    concate = Concatenate(axis=-1)([input_f, input_s])
    
    print(concate.shape)
    branch0 = self.conv_bn_act(concate)
    print(branch0.shape)
    branch_1 = self.conv_act1(branch0)
    
    branch_1 = self.conv_act2(branch_1)
    
    x = multiply([branch0, branch_1])
    return  Add()([branch0, x])


In [0]:
norm1 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
norm2 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = FeatureFusionModule(n_filters=64)
block(norm1, norm2)

print([x.name for x in block.variables])

(TensorShape([Dimension(10), Dimension(256), Dimension(256), Dimension(64)]), TensorShape([Dimension(10), Dimension(256), Dimension(256), Dimension(64)]))
(10, 256, 256, 128)
(10, 127, 127, 64)
['feature_fusion_module/conv_bn_act_2/Conv2D/kernel:0', 'feature_fusion_module/conv_bn_act_2/Conv2D/bias:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/gamma:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/beta:0', 'feature_fusion_module/conv_act_1/Conv2D/kernel:0', 'feature_fusion_module/conv_act_1/Conv2D/bias:0', 'feature_fusion_module/conv_act_2/Conv2D/kernel:0', 'feature_fusion_module/conv_act_2/Conv2D/bias:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/moving_mean:0', 'feature_fusion_module/conv_bn_act_2/BatchNorm/moving_variance:0']


In [0]:
norm1 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
print(GlobalMaxPooling2D()(norm1).shape)


(10, 64)


In [0]:
class ContextPath(tf.keras.Model):
  def __init__(self, n_filters):
    super(ContextPath, self).__init__(name='')
    
    self.globalmax = GlobalMaxPooling2D()
    self.upsample_1 = UpSampling2D(size=(2, 2))
    self.upsample_2 = UpSampling2D(size=(2, 2))
    self.upsample_3 = UpSampling2D(size=(2, 2))
    self.upsample_4 = UpSampling2D(size=(2, 2))
    self.arm1 = AttentionRefinmentModule(n_filters=64)
    self.arm2 = AttentionRefinmentModule(n_filters=64)

  def call(self, tail_layer):
    globalmax = self.globalmax(tail_layer)
    upsample1 = self.upsample_1(tail_layer)
    upsample2 = self.upsample_2(upsample1)
    upsample3 = self.upsample_3(upsample2)
    upsample4 = self.upsample_4(upsample3)

    seg1 = self.arm1(upsample3)
    seg2 = self.arm2(upsample4)


    tail = Add()([upsample4, globalmax])

    cnc = Concatenate(axis=-1)([tail,seg2])

    final_feature = Concatenate(axis=-1)([cnc, seg2])
    return final_feature
    
      
      

In [0]:
tail_layer = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
block = ContextPath(n_filters=64)
block(tail_layer)


(10, 2048, 2048, 64)
(10, 2048, 2048, 64)
(10, 4096, 4096, 64)
(10, 4096, 4096, 64)


<tf.Tensor 'context_path/concatenate_2/concat:0' shape=(10, 4096, 4096, 192) dtype=float32>

In [0]:
class FinalModel(tf.keras.Model):
  def __init__(self):
    self.context_path = ContextPath(64)
    self.batch1 = ConvBnAct(64)
    self.batch2 = ConvBnAct(128)
    self.batch3 = ConvBnAct(256)
    self.upsample = UpSampling2D(size=(8, 8))
    self.feature_fusion = FeatureFusionModel(64)
    
  def call(self, x, last_layer): 
    
    #spatial path
    x = self.batch1(x)
    x = self.batch2(x)
    x = self.batch3(x)
    
    #context path
    cp = ContextPath(last_layer)
    
    #concatenate
    ffm = Concatenate(axis=-1)([cp,x])
    
    #upsample
    ans = self.upsample(ffm)
    
    return ans
    


In [0]:
norm1 = tf.random_normal([10, 256,256], mean=-1, stddev=4)


In [0]:
tk = Conv1D(filters=220, kernel_size=(5), strides=2)(norm1)

In [0]:
print(tk.shape)

(10, 126, 256)


In [0]:
tensor = tf.random_normal([100,2000, 50], mean=-1, stddev=4)

In [0]:
s=TimeDistributed(Dense(20), name='embedding_dropout')(tensor)
print(s.shape)

(100, 2000, 20)


In [0]:
print(s)

Tensor("embedding_dropout/transpose_1:0", shape=(100, 2000, 20), dtype=float32)


In [0]:
s=GlobalMaxPooling1D()(tensor)
print(s.shape)

(100, 50)


In [0]:
#https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
#https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras

In [0]:
s = tf.random_normal([100, 100], mean=-1, stddev=4)

keras.layers.RepeatVector(6)(s)

<tf.Tensor 'repeat_vector/Tile:0' shape=(100, 6, 100) dtype=float32>

In [0]:
tf.keras.backend.expand_dims(
    s,
    axis=2
)


<tf.Tensor 'ExpandDims:0' shape=(100, 100, 1) dtype=float32>

In [0]:
s1 = tf.random_normal([100, 100], mean=-1, stddev=4)

s2 = tf.random_normal([100, 100], mean=-1, stddev=4)

(s1<s2)

<tf.Tensor 'Less:0' shape=(100, 100) dtype=bool>

In [0]:
#ako sakam da vrzam RNN rezuoltati so softmax => TimeDistributed, vo novite vrZii samo dense
# ako sakam da vrzam CNN ogromen layer praes global max pooling

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
print(x.shape)
x=GlobalAveragePooling2D()(x)
print(x.shape)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(120,activation='softmax')(x) #final layer with softmax activation

NameError: ignored

In [0]:
s2 = tf.random_normal([20,50,50,40], mean=-1, stddev=4)
keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(s2) #'valid/same'

<tf.Tensor 'max_pooling2d_9/MaxPool:0' shape=(20, 25, 25, 40) dtype=float32>

In [0]:
from keras.models import Sequential
## create model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


NameError: ignored